Misc derivations of frame2d FEM elements and verification that results are consistent with welib.FEM.frame2d

In [ ]:
import sympy
from sympy import symbols, Symbol, simplify, Function, expand_trig
from sympy import cos,sin
from sympy import diff, integrate
from sympy import python, latex
from sympy import Matrix
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
# --- local packages
import welib.FEM
from welib.FEM.frame2d import *
from welib.FEM.fem_sympy import *

In [ ]:
s = Symbol('s')
x = Symbol('x')
L = Symbol('L')
EI=Symbol('EI') 
EA=Symbol('EA') 
rho = Symbol('rho')
A = Symbol('A')
m = Symbol('m')
T=Symbol('T')   # Axial load
u1,u2,u3,u4,u5,u6 = symbols('u_1, u_2, u_3, u_4, u_5, u_6')

In [ ]:
# --- Visualize shape functions
vx = np.linspace(0,1,100)
fig,ax = plt.subplots(1,1)
ax.plot(vx, b2(vx),   label=r'$b_2$')
ax.plot(vx, b3(vx,1), label=r'$b_3$')
ax.plot(vx, b5(vx),   label=r'$b_5$')
ax.plot(vx, b6(vx,1), label=r'$b_6$')
fig.legend()

fig,ax = plt.subplots(1,1)
ax.plot(vx, h(vx,1,2,3,-5,1),   label=r'$h(x)\ for\ u=(1,2,3,-5)$')
fig.legend()

fig,ax = plt.subplots(1,1)
ax.plot(vx, uax(vx,1,2,1),   label=r'$u(x)\ for\ u=(1,2)$')
fig.legend()



In [ ]:
# --- Look at properties
print('h in polynomial form')
h(x, u2, u3,u5,u6,L).simplify().expand().collect(x)
print('u in polynomial form')
uax(x, u1, u4, L).simplify().expand().collect(x)

D0 = []
DL = []
V0 = []
VL = []
for b in [b1, b2, lambda s:b3(s,L), b4, b5, lambda s:b6(s,L)]:
    D0.append(b(s).diff(s).subs([(s,0)]))
    DL.append(b(s).diff(s).subs([(s,1)]))
    V0.append(b(0))
    VL.append(b(1))
#print('Values at start points ')
#V0
#print('Values at end points ')
#VL
print('Derivatives at start points ')
D0
print('Derivatives at end points ')
DL

print('Integrals of b functions')
I = []
for b in [b1, b2, lambda s:b3(s,L), b4, b5, lambda s:b6(s,L)]:
    I.append(integrate(b(s),(s,0,1)))
I

In [ ]:
# --- Define Stiffness matrix based on Strain energy

UB = frame2d_U_B(h,x,L,EI)
UG = frame2d_U_G(h,x,L,T)
UA = frame2d_U_A(uax,x,L,EA)
#UB.simplify()

Ke= zeros(6,6)

u=[u1,u2,u3,u4,u5,u6]
            
Ke= stiffnessMatrixFromPot(UB+UA, u)
Kg= stiffnessMatrixFromPot(UG, u)
            
Ke
Kg

print(python(Kg))
print(python(Ke))

In [ ]:
# --- Interpolation matrix from nodal DOF to deflections
NN = zeros(2,6)
NN[0,0]=b1(x/L)
NN[0,3]=b4(x/L)
NN[1,1]=b2(x/L)
NN[1,2]=b3(x/L,L)
NN[1,4]=b5(x/L)
NN[1,5]=b6(x/L,L)

NN

Ke = stiffnessMatrixFromShapeFunctions(NN, x, L, EI) # TODO missing EA
Ke1,Me1,Kg1=frame2d_KeMe(EA, EI, L, m,T)
Ke1=Matrix(Ke1)
Ke-Ke1

In [ ]:
# --- Mass matrix

Me= massMatrixFromShapeFunctions(NN, x, L, m)
Ke1,Me1,Kg1=frame2d_KeMe(EA, EI, L, m,T)

Me1=Matrix(Me1)
Me1-Me


Me
print(python(Me))

In [ ]:
# --- latex export
print(cleantex(latex(Me/m)))
print(cleantex(latex(Matrix(Ke1))))
print(cleantex(latex(Matrix(Kg)/T)))
print(cleantex(latex(Matrix(I))))